# Exploração de dados: EDA e pré-processamento
Giulia Chimini Stefainski, Leonardo Azzi Martins, Matheus de Moraes Costa

---

**Objetivo:** realizar uma análise exploratória de dados, e a partir disto definir possibilidades de pré-processamento para o dataset.

# Setup

In [ ]:
%pip install torch==2.6.0+cu124 \
  --index-url https://download.pytorch.org/whl/cu124


In [ ]:
%pip install pandas==1.5.3 transformers==4.50.2 datasets==3.5.0 scikit-learn==1.4.2 evaluate==0.4.3 seaborn==0.13.2 imblearn accelerate==1.5.2 emoji==2.14.1

In [ ]:
%pip install --upgrade numpy transformers

In [ ]:
%pip install --force-reinstall --upgrade numpy pandas scikit-learn torch transformers

In [ ]:
from transformers import Trainer
from datasets import Dataset
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch
import emoji

# Preparação de dados
Carrega o dataset a ser utilizado para fine-tuning e seleciona os atributos mais relevantes.

Faz o download do dataset anotado no diretório ./data

In [ ]:
import os

if not os.path.exists('./data/covidbr_labeled.csv'):
  %mkdir data
  %curl -L -o ./data/covidbr_labeled.csv https://zenodo.org/records/5193932/files/covidbr_labeled.csv
else:
    print("File already exists. Skipping download.")

In [ ]:
original_dataset_df = pd.read_csv('./data/covidbr_labeled.csv')
original_dataset_df

In [ ]:
dataset_df = original_dataset_df[["text", "misinformation"]]
dataset_df

# Análise exploratória de dados

O objetivo é entender melhor e sumarizar as características dos dados, analisando quantidade e tipos de atributos, verificando distribuição do atributo alvo, identificando padrões e anomalias, removendo atributos que pareçam irrelevantes ou problemáticos, etc. Utilize gráficos e sumarizações estatísticas para a EDA. Verifique potenciais problemas nos dados, como por exemplo, a necessidade de normalizar os atributos, balancear classes, ou remover instâncias ou atributos por inconsistências nos dados.

- P1. Qual a quantidade e tipos de atributos? Existem inconsistências?
  - Quais são os atributos disponíveis?
  - Existem inconsistências nos atributos? (Atributos vazios, potenciais erros, etc)
  - Existem atributos que necessitam ser removidos ou transformados?
- P2. Existem padrões e anomalias nos dados?
  - Existem tendências ou inconsistências nos atributos? (Atributos vazios, potenciais erros, etc)
- P3. Qual a distribuição do atributo alvo?
  - Quais são as classes alvo? Qual a distribuição entre as classes? Está balanceada ou desbalanceada?


In [ ]:
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker

## P1. Qual a quantidade e tipos de atributos? Existem inconsistências?

### 1.1 Quais são os atributos disponíveis?

In [ ]:
original_dataset_df.info()

In [ ]:
dataset_df.info()

## P2. Existem padrões e anomalias nos dados?
Existem tendências ou inconsistências nos atributos? (Atributos vazios, potenciais erros, etc)

### 3.1 Quais são os padrões?

In [ ]:
# Tokeniza o atributo 'text' de dataset_df por classe de misinformation usando whitespaces
for label in sorted(dataset_df['misinformation'].unique()):
    print(f"\nClasse misinformation = {label}")
    texts = dataset_df[dataset_df['misinformation'] == label]['text']
    tokenized = texts.apply(lambda x: str(x).split())
    token_lengths = tokenized.apply(len)
    print(token_lengths.describe())
    print(f"Mediana do comprimento dos tokens: {token_lengths.median()}")

    words = texts.apply(lambda x: str(x).split())
    word_lengths = words.apply(lambda ws: [len(w) for w in ws if len(w) > 0])
    all_word_lengths = [l for sublist in word_lengths for l in sublist]
    avg_word_length = np.mean(all_word_lengths) if all_word_lengths else 0
    print(f"Tamanho médio de palavras (em caracteres) para classe {label}: {avg_word_length:.2f}")

plt.figure(figsize=(10, 6))
plt.hist(token_lengths, bins=50, edgecolor='black', log=True)
plt.title('Distribuição do número de tokens do atributo text (escala log)')
plt.xlabel('Número de tokens')
plt.ylabel('Frequência (escala log)')
plt.show()

# Boxplot do número de tokens por texto, separado por classe de misinformation
plt.figure(figsize=(10, 6))
token_lengths_by_class = [
    dataset_df[dataset_df['misinformation'] == label]['text'].apply(lambda x: len(str(x).split()))
    for label in sorted(dataset_df['misinformation'].unique())
]
plt.boxplot(token_lengths_by_class, vert=True, patch_artist=True,
            boxprops=dict(facecolor='white', color='black', linewidth=1.5),
            medianprops=dict(color='red', linewidth=2),
            whiskerprops=dict(color='black', linewidth=1.5),
            capprops=dict(color='black', linewidth=1.5),
            flierprops=dict(marker='o', markerfacecolor='gray', markersize=4, alpha=0.4, markeredgecolor='black'),
            widths=0.5)
plt.yscale('log')
plt.title('Box-plot do número de tokens por texto para cada classe (escala log)')
plt.xlabel('Classe de misinformation', fontsize=14)
plt.ylabel('Número de tokens (escala log)', fontsize=14)
plt.xticks([1, 2], sorted(dataset_df['misinformation'].unique()))
plt.grid(axis='y', linestyle='--', alpha=0.6, which='both')
plt.gca().yaxis.set_major_formatter(mticker.ScalarFormatter())

plt.tight_layout()
plt.show()

### 3.2 Existem inconsistências nos atributos? (Atributos vazios, potenciais erros, etc)

#### 3.2.1 Valores nulos

In [ ]:
dataset_df[dataset_df.isnull().any(axis=1)]

Remove instância com texto nulo, pois é irrelevante para o treinamento

In [ ]:
dataset_df = dataset_df.dropna()
dataset_df.isnull().any()

#### 3.2.2 URLs

In [ ]:
import re 
# Busca por textos que contém qualquer ocorrência de URLs
any_url_pattern = r'(https?://[^\s]+|www\.[^\s]+|\b[^\s]+?\.(com|br|org|net|gov|edu|pt)\b)'

# Busca por textos que começam com URLs
start_url_pattern = r'^(https?://[^\s]+|www\.[^\s]+|\b[^\s]+?\.(com|br|org|net|gov|edu|pt)\b)'

# Busca por textos que contém exclusivamente URLs
only_url_pattern = r'^(https?://[^\s]+|www\.[^\s]+|\b[^\s]+?\.(com|br|org|net|gov|edu|pt))$'

##### Busca por textos que começam com URLs

Busca instâncias de text onde começa com uma URL. Conforme Martins et al. 2021, estas instâncias podem dificultar a classificação, resultando em um ganho de aprox. 10% em F1-score ao remover estas instâncias.

In [ ]:
start_url_df = dataset_df[dataset_df['text'].str.contains(start_url_pattern, na=False)]
start_url_df

##### Busca por textos que contém qualquer ocorrência de URLs

In [ ]:
url_df = dataset_df.copy()
url_df = url_df[url_df['text'].str.contains(any_url_pattern, regex=True, flags=re.IGNORECASE)]
url_df

In [ ]:
url_df[url_df['misinformation'] == 0].count()

In [ ]:
url_df[url_df['misinformation'] == 1].count()

##### Busca por textos que **não** contém ocorrências de URLs

In [ ]:
no_url_df = dataset_df.copy()
no_url_df = no_url_df[~no_url_df['text'].str.contains(any_url_pattern, regex=True, flags=re.IGNORECASE)]
no_url_df

In [ ]:
no_url_df[no_url_df['misinformation'] == 0].count()

In [ ]:
no_url_df[no_url_df['misinformation'] == 1].count()

##### Busca por textos que contém **exclusivamente** URLs

In [ ]:
only_url_rows = dataset_df[dataset_df['text'].str.match(only_url_pattern, na=False)]
only_url_rows

In [ ]:
only_url_rows[only_url_rows['misinformation'] == 0].count()

In [ ]:
only_url_rows[only_url_rows['misinformation'] == 1].count()

##### Existe o mesmo dataset filtrado conforme Martins et al. (2021)?

Reproduz o notebook de Martins et al. (2021), que reporta existirem 1.509 mensagens com apenas URLs como conteúdo em texto, divergindo do nosso achado de 498 instâncias.

In [ ]:
import re
martins_df = dataset_df.copy()
martins_df['cleanLinks'] = martins_df['text'].apply(lambda x: re.split(r'http:\/\/.*', str(x))[0])
martins_df

In [ ]:
martins_df['cleanLinks']

In [ ]:
martins_df[martins_df['cleanLinks'] != '' ].shape

A metodologia aplicada não foi capaz de filtrar corretamente as mensagens exclusivamente compostas por URL.

##### Distribuição

In [ ]:
import seaborn as sns

labels = ['Contém URL', 'Sem URL', 'Apenas URL']

tipo = []
classe = []
for df, nome in zip([url_df, no_url_df, only_url_rows], labels):
    tipo.extend([nome] * len(df))
    # Use the correct column name for the class label
    if 'labels' in df.columns:
        classe.extend(df['labels'].tolist())
    else:
        classe.extend(df['misinformation'].tolist())

plt.figure(figsize=(10, 6))
sns.countplot(x=tipo, hue=classe, palette=['#377eb8', '#e41a1c'])
plt.title('Distribuição das classes por tipo de mensagem')
plt.xlabel('Tipo de mensagem')
plt.ylabel('Quantidade')
plt.legend(title='Classe', labels=['Não desinformação', 'Desinformação'])

plt.show()


#### 3.2.3 Emojis 🤠

##### Busca mensagens compostas por emojis
- `emoji_count`: conta a quantidade de emojis em 'text'
- `emoji_ratio`: calcula a taxa de emojis por mensagem

In [ ]:
def count_emojis(text):
    return sum(1 for char in text if char in emoji.EMOJI_DATA)

def char_count(text):
    return len(text)

def word_count(text):
    return len(text.split())

dataset_df['emoji_count'] = dataset_df['text'].apply(count_emojis)
dataset_df['char_count'] = dataset_df['text'].apply(char_count)
dataset_df['word_count'] = dataset_df['text'].apply(word_count)

def emoji_ratio(text):
    return count_emojis(text) / char_count(text) if char_count(text) > 0 else 0

dataset_df['emoji_ratio'] = dataset_df['text'].apply(emoji_ratio)

dataset_df

##### Contagem de emojis

In [ ]:
display(dataset_df.sort_values(by='emoji_count', ascending=False).reset_index(drop=True))

In [ ]:
dataset_df['emoji_count'].describe()

In [ ]:
dataset_df[dataset_df['misinformation'] == 0]['emoji_count'].describe()

In [ ]:
dataset_df[dataset_df['misinformation'] == 1]['emoji_count'].describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.boxplot(x='misinformation', y='emoji_count', data=dataset_df, palette=['#377eb8', '#e41a1c'])
plt.title('Box-plot do número de emojis (escala log)')
plt.xlabel('Desinformação')
plt.ylabel('Número de emojis (log)')
plt.yscale('log')   
plt.gca().yaxis.set_major_formatter(mticker.ScalarFormatter())

plt.legend(title='Classe', labels=['Não desinformação', 'Desinformação'])

plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(dataset_df['emoji_count'], edgecolor='black', bins=range(0, int(dataset_df['emoji_count'].max()) + 25, 25), alpha=0.7)
plt.title('Histograma do número de emojis por mensagem (escala log)')
plt.xlabel('Número de emojis por mensagem')
plt.ylabel('Frequência (log)')
plt.yscale('log')
plt.xticks(range(0, int(dataset_df['emoji_count'].max()) + 1, 25))
plt.show()

##### Taxa de emojis por mensagem

Verifica as instâncias com maior emoji_ratio

In [ ]:
display(dataset_df.sort_values(by='emoji_ratio', ascending=False).reset_index(drop=True))

In [ ]:
dataset_df['emoji_ratio'].describe()

Existe alguma relação entre a taxa de emojis e o atributo preditivo?

In [ ]:
dataset_df[dataset_df['misinformation'] == 0]['emoji_ratio'].describe()

In [ ]:
dataset_df[dataset_df['misinformation'] == 1]['emoji_ratio'].describe()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='misinformation', y='emoji_ratio', data=dataset_df, palette=['#377eb8', '#e41a1c'])
plt.title('Box-plot da taxa de emojis por caractere da mensagem (escala log)')
plt.xlabel('Desinformação')
plt.ylabel('Taxa de emojis por caractere (log)')
plt.yscale('log')
plt.ylim(0, 1)
plt.gca().yaxis.set_major_formatter(mticker.ScalarFormatter())
plt.legend(title='Classe', labels=['Não desinformação', 'Desinformação'])
plt.show()

Verifica as instâncias com maior emoji_count

In [ ]:
display(dataset_df.sort_values(by='emoji_count', ascending=False).reset_index(drop=True))

Não existem textos com emoji ratio maior que ~0.02. Portanto, não precisam ser tratados.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(dataset_df['emoji_ratio'], edgecolor='black')
plt.title('Histograma da taxa de emojis por caractere (escala log)')
plt.xlabel('Taxa de emojis por caractere')
plt.ylabel('Frequência (log)')
plt.yscale('log')
plt.show()

## P2. Qual a distribuição do atributo alvo?

In [ ]:
import seaborn as sns

series = dataset_df['misinformation'].value_counts()
print(series)

fig = plt.figure(figsize=(6, 5))

ax = sns.countplot(
    x=dataset_df['misinformation'],
    data=dataset_df,
    hue='misinformation',
    palette=['#377eb8', '#e41a1c'],
    order=dataset_df['misinformation'].value_counts().index
)
plt.legend(title='Classe', labels=['Não desinformação', 'Desinformação'])

Isto indica que o dataset está desbalanceado, fator que pode enviesar o treinamento.

**E qual a distribuição removendo instâncias que contém URLs?**

In [ ]:
import seaborn as sns

series = no_url_df['misinformation'].value_counts()

print(series)

fig = plt.figure(figsize=(5, 3))

sns.countplot(x=no_url_df['misinformation'], data = no_url_df,
              hue='misinformation', palette=['#377eb8', '#e41a1c'],
              order=dataset_df['misinformation'].value_counts().index
)

# Pré-processamento

In [ ]:
# Configurações iniciais
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando o dispositivo: {device}")

## Limpeza

In [ ]:
dataset_df.info()

Dataset removendo instâncias onde o texto contém URLs em seu início

In [ ]:
no_start_url_df = dataset_df[~dataset_df['text'].str.contains(r'^(http|www)', na=False)].reset_index(drop=True)
no_start_url_df.info()

Dataset removendo todas as URLs do texto

In [ ]:
no_url_df.info()

## Labels
O HuggingFace Trainer utiliza o rótulo labels para identificar os rótulos no treinamento. Renomeando a coluna alvo para 'labels'

In [ ]:
dataset_df = dataset_df.rename(columns={'misinformation': 'labels'})
no_start_url_df = no_start_url_df.rename(columns={'misinformation': 'labels'})
no_url_df = no_url_df.rename(columns={'misinformation': 'labels'})

## Tokenização

Carrega o tokenizador para `bert-base-portuguese-cased` (BERTimbau)

In [ ]:
from transformers import AutoTokenizer  # Or BertTokenizer

hf_model_name = 'neuralmind/bert-base-portuguese-cased'
tokenizer = AutoTokenizer.from_pretrained(hf_model_name, do_lower_case=False)

Criamos uma função de tokenização, que será utilizada para tokenizar cada valor de um Pandas DataFrame em forma de função de mapeamento.

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

In [ ]:
def tokenize(df):
  dataset = Dataset.from_pandas(df)
  dataset_tk = dataset.map(tokenize_function, batched=True, remove_columns=['text']) #'__index_level_0__'
  return dataset_tk

## Balanceamento de classes

Dado que o dataset tem sua classe misinformation desbalanceada, utilizou-se o método de cálculo de class_weights, que atribui pesos na função loss do treinador para 'compensar' o desbalanceamento.

"If "balanced", class weights will be given by `n_samples / (n_classes * np.bincount(y=labels))`. If a dictionary is given, keys are classes and values are corresponding class weights. If None is given, the class weights will be uniform."

Referências:
- https://medium.com/@heyamit10/fine-tuning-bert-for-classification-a-practical-guide-b8c1c56f252c
- https://discuss.huggingface.co/t/class-weights-for-bertforsequenceclassification/1674

get_class_weights(df):
- Cria uma instância do CrossEntropyLoss com os pesos calculados
- Recria a classe WeightedTrainer para 'sobrescrever' a classe original no HuggingFace Trainer, utilizada a computação do loss ponderada configurada acima.

In [ ]:
def get_class_weights(df):
  labels = df["labels"]

  class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)

  class_weights = torch.tensor(class_weights, dtype=torch.float)

  print(class_weights)

  loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))

  class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss
      
  return WeightedTrainer

In [ ]:
# Teste
get_class_weights(dataset_df)